In [9]:
%reload_ext autoreload
%autoreload 2
import logging
import grpc
import pandas as pd
import sys
sys.path.append('proto')
from proto import contest_pb2
from proto import contest_pb2_grpc
from proto import question_pb2
from proto import question_pb2_grpc
import numpy as np
import time
from utils import *
from strategy_utils import *

In [10]:
contest_channel=grpc.insecure_channel('47.103.23.116: 56702')
question_channel=grpc.insecure_channel('47.103.23.116: 56701')

contest_stub=contest_pb2_grpc.ContestStub(contest_channel)
question_stub=question_pb2_grpc.QuestionStub(question_channel)

login_response=contest_stub.login(contest_pb2.LoginRequest(user_id=88,user_pin='dDTSvdwk'))

session_key=login_response.session_key
init_capital=login_response.init_capital

In [11]:
i=0#控制seq
count=0
data_lst=[]
factors_lst=[]
period=1#eg 每两天跑一次策略
comission=0#
max_exposure=0.1#大盘上涨，多头增加，大盘下跌，空头增加
single_stock_position_limit=0.1#
lending_rate=0.01#
borrow_rate=0.05#
leverage=2#
factors=pd.DataFrame()
prev_factors=pd.DataFrame()#上一次计算的factors

In [13]:
while True:
    login_response=contest_stub.login(contest_pb2.LoginRequest(user_id=88,user_pin='dDTSvdwk'))

    session_key=login_response.session_key
    init_capital=login_response.init_capital
    i = 0
    try:
        while True:
            question_response=question_stub.get_question(question_pb2.QuestionRequest(user_id=88,sequence=i))
            print(question_response.sequence)
            if question_response.sequence!=-1:
                dailystk = [x.values for x in question_response.dailystk]
                data_lst.extend(dailystk)

                if count>10:#开始不动，只要有新数据就跑一次策略
                    print('run strategy')
                    df=pd.DataFrame(data_lst,columns=['day','stock','open','high','low','close','volume'],
                                    dtype=float).set_index(['day','stock'])
                    
                    dailyfactors=get_factors(df,prev_factors)  #从数据获取因子
                    prev_factors=dailyfactors

                    factors_lst.extend(dailyfactors.values)#向因子库追加
                    #部份因子可能需要根据历史因子数据归一化

                    # factor_select=select_factors(factors,n=10,period=period)  #计算相关系数选取因子
                    # factor_select=['avg', 'mom', 'max52', 'cci', 'K', 'D', 'J', 'rsi', 'trix', 'willr']
                    # factor_select=['avg','mom', 'vol', 'max52', 'K', 'D', 'J', 'rsi', 'trix', 'willr', 'macd', 'natr','mfi']
                    factor_select=['avg', 'mom', 'max52', 'D', 'willr', 'natr', 'rsi']

                    # index_direction='neutral'#TODO 大盘方向，用于控制exposure

                    weights = get_weight(dailyfactors[factor_select],head_n=5,tail_n=5)
                    if count < 100:
                        target_pos=get_position(weights,
                                                pd.DataFrame(dailystk,columns=['day','stock','open','high','low','close','volume'],dtype=float),#只需要close，待优化
                                                question_response.positions,
                                                question_response.capital / 2,
                                                comission)
                    else:
                        target_pos=get_position(weights,
                                                pd.DataFrame(dailystk,columns=['day','stock','open','high','low','close','volume'],dtype=float),#只需要close，待优化
                                                question_response.positions,
                                                question_response.capital,
                                                comission)
                    
                    #提交策略
                    if count%period==0:#按周期提交
                        ##summit answer
                        submit_response = contest_stub.submit_answer(contest_pb2.AnswerRequest(user_id=88,user_pin='dDTSvdwk',session_key=login_response.session_key,sequence=i,positions=target_pos))

                        print(submit_response,question_response.capital)
                        if not submit_response.accepted:
                            print(submit_response.reason)
                            if submit_response.reason[-7:] == 'timeout':
                                i=question_response.sequence+1
                                count+=1
                                continue#如果提交超时，直接请求新数据
                
                i=question_response.sequence+1
                count+=1
            time.sleep(1)
            # if count==20:
                # break
    except Exception as e:
        print(e)
    time.sleep(1)

731
run strategy
stock
1001.0     -74488
1002.0    -170160
1003.0    -141124
1004.0     -15747
1014.0     307436
1023.0     290551
1037.0     -16472
1038.0    -178356
1060.0    -884577
1062.0    2626691
1063.0     500740
1080.0     512744
1081.0     -94253
1084.0    -289993
1097.0      49229
1105.0     207456
1112.0     436632
1133.0     -38720
1135.0    -159539
1150.0    2072842
1153.0    4081826
1162.0    -320099
1165.0    3997901
1166.0    -536795
1180.0      -3211
1181.0      -3474
1185.0    -474734
1193.0     609118
1194.0    1470740
1205.0    -823362
1216.0      -4160
1221.0    -123492
1232.0     623060
1235.0     195178
1244.0    -161844
1246.0      -3929
1251.0    1936887
1253.0      79008
1270.0      -3641
1280.0     162405
1289.0     -89629
1300.0     -23218
1304.0     542960
1306.0     -22113
1327.0    2520562
1334.0      27726
1336.0     118310
Name: close, dtype: int32
reason: "FATAL ERROR: User_id:88 Sequence:0 answer timeout"
 573133547.1576294
FATAL ERROR: User_id:88 Se

KeyboardInterrupt: 

In [20]:
question_response.dailystk

AttributeError: 'google.protobuf.pyext._message.RepeatedCompositeCo' object has no attribute 'values'

In [24]:
daily = np.asarray([x.values for x in question_response.dailystk])

In [15]:
question_response=question_stub.get_question(question_pb2.QuestionRequest(user_id=88,sequence=i))

In [25]:
daily[:,2]

array([  52.87062,  127.3008 ,   60.0468 ,   52.833  ,  589.3074 ,
        289.5552 ,   56.958  ,  470.7516 ,   19.206  ,   60.0402 ,
         26.3868 ,  452.925  ,  347.3052 ,  184.4832 ,   32.01   ,
         85.4337 ,  235.2702 ,  144.8832 ,  570.075  ,   69.4386 ,
        119.79   ,  320.3178 ,  422.3934 ,   46.0944 ,   45.3486 ,
       1053.36   ,  391.8552 ,   77.55   ,    8.283  ,  120.8394 ,
         39.0258 ,  284.856  ,  495.99   ,  789.15606,  117.711  ,
        352.572  ,  676.17   ,  571.4082 ,   45.6918 ,  368.3922 ,
         84.1962 ,   98.34   ,   91.2318 ,   77.0352 ,  351.6414 ,
        632.61   ,  485.1    ,  567.336  ,   34.5444 ,   35.079  ,
         43.6722 ,   45.8502 ,  174.834  ,   26.2878 ,  221.1    ,
        191.5056 ,  142.0518 ,    9.7746 ,   60.0402 ,   68.3628 ,
         38.8014 ,   46.5168 ,    8.91   ,   19.6152 ,   89.958  ,
         12.87   ,   13.86924, 1420.98   ,  190.476  ,   10.35276,
        128.139  ,  355.014  ,   83.3712 ,  508.2    ,  243.54